In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from collections.abc import MutableMapping
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
shelter = AnimalShelter()

# create dataframe from mongoDB query method. filter data by removing '_id' and '' columns
df = pd.DataFrame.from_records(shelter.read({})).drop(['_id'], axis=1)

#########################
# Dashboard Layout / View
#########################

# create dash application
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvare.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Place the HTML image tag in the line below into the app.layout code according to your design
#Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('James Soto Project 2 Dashboard'))),
    
    
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),

    # div to hold radio items for filtering by resuce type
    html.Div([
        html.Center(dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'Remove all filters'}
            ],
            value='Filter table by rescue dog types',
            inputStyle={"margin-left": "20px"},
            labelStyle={'display': 'inline-block'}
        ))
    ]),
    html.Br(),

    # create the data table
    dash_table.DataTable(
        id='datatable-id',

        # for every column, set the name and id. also set column options
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],

        # populate table with records from DataFrame
        data=df.to_dict('records'),

        # set data table interactivity
        editable=False,
        filter_action="native",
        sort_action='custom',
        sort_mode="multi",
        sort_by=[],
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),

    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex', "width": "500"},
             children=[
                 html.Div(
                     className='row',
                     style={'display': 'flex'},
                     children=[
                         # where the histogram will go
                         html.Div(
                             dcc.Graph(id='graph-id'),
                             className='col s12 m6',
                         ),
                         # where the map will go
                         html.Div(
                             id='map-id',
                             className='col s12 m6',
                         ),
                     ]
                 ),
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-id', 'data'),
    [Input(component_id='radio-items', component_property='value')
     ])
def radioFilter(radio_options):
    """ functionality for radio items. update data table with rescue type filters """
    if radio_options == 'Water Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'})).drop(['_id'], axis=1)
        dff = dff.loc[(dff['breed'] == 'Labrador Retriever Mix') |
                      (dff['breed'] == 'Chesapeake Bay Retriever') |
                      (dff['breed'] == 'Newfoundland')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Female')]
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')

    elif radio_options == 'Mountain Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'})).drop(['_id'], axis=1)
        dff = dff.loc[(dff['breed'] == 'German Shepherd') |
                      (dff['breed'] == 'Alaskan Malamute') |
                      (dff['breed'] == 'Old English Sheepdog') |
                      (dff['breed'] == 'Siberian Husky') |
                      (dff['breed'] == 'Rottweiler')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Male')]
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')

    elif radio_options == 'Disaster Rescue':
        dff = pd.DataFrame.from_records(shelter.read({})).drop(['_id'], axis=1)
        dff = dff.loc[(dff['breed'] == 'German Shepherd') |
                      (dff['breed'] == 'Doberman Pinscher') |
                      (dff['breed'] == 'Golden Retriever') |
                      (dff['breed'] == 'Bloodhound') |
                      (dff['breed'] == 'Rottweiler')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Male')]
        dff = dff.query('20 <= age_upon_outcome_in_weeks <= 300')

    else:
        dff = pd.DataFrame.from_records(shelter.read({})).drop(['_id'], axis=1)

    return dff.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', 'figure'),
    [Input('datatable-id', "derived_viewport_data"),
    ])
#Histogram filtered by breed in the data table
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    figure = px.histogram(dff, x=['breed'])
    return figure
            
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, index):
    if viewData is None:
        return []
    else:
        # Add in the code for your geolocation chart
        dff = pd.DataFrame.from_dict(viewData)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0, 13], dff.iloc[0, 14]], zoom=10,
                   children=[         
                       dl.TileLayer(id="base-layer-id"),
                       # Marker with tool tip and popup
                       dl.Marker(position=[dff.iloc[0, 13], dff.iloc[0, 14]], children=[    
                           dl.Tooltip(dff.iloc[0, 4]),
                           dl.Popup([
                               html.H1(dff.iloc[0, 9]),
                               html.P(dff.iloc[0, 8])
                            
                           ])
                       ])
                   ])
        ]

# run the application
app.run_server(mode='inline', port = 8094)

Dash is running on http://127.0.0.1:8094/

